In [83]:
import matlab.engine
import numpy as np
import os

In [84]:
 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

In [85]:
import os
from pathlib import Path

# Base directory (adjust to your setup)
base_dir = Path("./")

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_win64.exe"  # Adjust for your OS
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/.DS_Store'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/input'), PosixPath('SCOPE/output'), PosixPath('SCOPE/docs'), PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/.gitignore'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/.gitattributes'), PosixPath('SCOPE/run_scope_wrapper.m'), PosixPath('SCOPE/.git'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/src')]


In [86]:
def run_modtran(params, case_id):
    """Generates MODTRAN input file and runs simulation"""
    # Read template TP5 file
    with open(modtran_tp5_template, "r") as f:
        tp5_content = f.read()
    
    # Replace parameters in template
    tp5_content = tp5_content.replace("{H1}", f"{params['sensor_altitude']:.3f}")
    tp5_content = tp5_content.replace("{SZA}", f"{params['sza']:.1f}")
    tp5_content = tp5_content.replace("{VIS}", f"{params['visibility']:.2f}")
    # Add more parameter replacements as needed
    
    # Write case-specific TP5
    tp5_path = modtran_dir / f"case_{case_id}.tp5"
    with open(tp5_path, "w") as f:
        f.write(tp5_content)
    
    # Run MODTRAN
    subprocess.run([str(modtran_exe), str(tp5_path)], cwd=modtran_dir)
    
    # Parse output (using your MATLAB routines as reference)
    tp7_path = tp5_path.with_suffix(".tp7")
    return parse_tp7(tp7_path)  # Implement this based on your MATLAB code


In [87]:
import matlab.engine

class SCOPEWrapper:
    def __init__(self):
        # Use absolute path to avoid relative path issues
        self.base_dir = Path(__file__).parent.resolve()  # Assuming file is in project root
        self.scope_dir = self.base_dir / "SCOPE"
        
        # Verify directory structure
        if not self.scope_dir.exists():
            raise FileNotFoundError(
                f"SCOPE directory not found at {self.scope_dir}\n"
                f"Current working directory: {Path.cwd()}\n"
                f"Required structure:\n"
                f"{self.base_dir}/\n"
                f"├── SCOPE/\n"
                f"│   ├── run_scope_wrapper.m\n"
                f"│   ├── SCOPE.m\n"
                f"│   └── ...\n"
                f"└── YourScript.py"
            )

    def __enter__(self):
        self.eng = matlab.engine.start_matlab()
        self.eng.addpath(str(self.scope_dir), nargout=0)
        return self

    def __exit__(self, *args):
        self.eng.quit()
        
    def run(self, params):
        # Convert Python params to MATLAB types
        matlab_params = {
            'LAI': matlab.double([params['lai']]),
            'Cab': matlab.double([params['cab']]),
            'tts': matlab.double([params['sza']]),
            # Add more parameters
        }
        
        # Call your MATLAB wrapper function
        reflectance, fluorescence = self.eng.run_scope_wrapper(matlab_params, nargout=2)
        return np.array(reflectance), np.array(fluorescence)


In [34]:
def calculate_radiance(scope_reflectance, scope_fluorescence, modtran_results):
    """
    Calculate at-sensor radiance using:
    L = (E_sun * t1 + E_diff * t2) * ρ / (1 - ρ * s) + F * t3
    """
    # Extract MODTRAN components
    t1 = modtran_results['direct_transmittance']
    t2 = modtran_results['diffuse_transmittance']
    s = modtran_results['spherical_albedo']
    t3 = modtran_results['fluorescence_transmittance']
    
    # Calculate components
    reflected = (modtran_results['direct_irradiance'] * t1 + 
                modtran_results['diffuse_irradiance'] * t2) * scope_reflectance
    reflected /= (1 - scope_reflectance * s)
    
    fluoresced = scope_fluorescence * t3
    
    return reflected + fluoresced


In [35]:
def generate_synthetic_dataset(num_samples=1000):
    scope = SCOPEWrapper()
    
    # Parameter ranges (adjust based on your needs)
    param_ranges = {
        'lai': (0.5, 6.0),
        'cab': (20, 80),
        'sza': (20, 70),
        'visibility': (5, 50),
        'sensor_altitude': (350, 1500)
    }
    
    for i in range(num_samples):
        # Generate random parameters
        params = {
            'lai': np.random.uniform(*param_ranges['lai']),
            'cab': np.random.uniform(*param_ranges['cab']),
            'sza': np.random.uniform(*param_ranges['sza']),
            'visibility': np.random.uniform(*param_ranges['visibility']),
            'sensor_altitude': np.random.uniform(*param_ranges['sensor_altitude'])
        }
        
        # Run SCOPE
        reflectance, fluorescence = scope.run(params)
        
        # Run MODTRAN
        modtran_results = run_modtran(params, i)
        
        # Calculate radiance
        radiance = calculate_radiance(reflectance, fluorescence, modtran_results)
        
        # Create 17x17 patch (homogeneous for now)
        patch = np.tile(radiance, (17, 17, 1))
        
        # Save sample
        np.savez(
            output_dir / f"sample_{i:04d}.npz",
            radiance=patch,
            params=params,
            modtran=modtran_results
        )


In [36]:
def convolve_hyplant(radiance, wavelengths):
    """Convolve high-res radiance to HyPlant spectral response"""
    # Load HyPlant SRF (implement based on your data)
    srf = load_hyplant_srf()  
    
    # Perform convolution
    convolved = np.zeros(len(srf))
    for i, (center, fwhm) in enumerate(srf):
        # Create Gaussian filter
        sigma = fwhm / 2.3548
        weights = np.exp(-(wavelengths - center)**2 / (2*sigma**2))
        weights /= weights.sum()
        
        convolved[i] = np.sum(radiance * weights)
    
    return convolved


In [88]:
class SCOPEWrapper:
 def __init__(self):
     self.base_dir = Path("/Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project")  # Absolute path
     self.scope_dir = self.base_dir / "SCOPE"

     # Validate critical files (Corrected list)
     required_files = [
         self.scope_dir / "SCOPE.m",
         self.scope_dir / "run_scope_wrapper_json.m",
         self.scope_dir / "input" / "input_data.csv",
         self.scope_dir / "set_parameter_filenames.csv",  # Check this one
         # Removed setoptions.csv.  We don't *need* to check it explicitly.
     ]
     for f in required_files:
         if not f.exists():
             raise FileNotFoundError(f"Critical file missing: {f}")

 def __enter__(self):
     """Start MATLAB engine with proper paths and working directory"""
     self.eng = matlab.engine.start_matlab()
     self.eng.addpath(str(self.scope_dir), nargout=0)
     self.eng.addpath(str(self.scope_dir / "src"), nargout=0)
     self.eng.cd(str(self.scope_dir), nargout=0)  # Change to SCOPE directory!
     return self

 def __exit__(self, exc_type, exc_val, exc_tb):
     """Ensure MATLAB engine stops"""
     if self.eng:
         self.eng.quit()

 def run(self, params):
     """Execute SCOPE simulation with parameter updates"""
     with tempfile.TemporaryDirectory() as tmpdir:
         tmp_path = Path(tmpdir)
         input_copy = tmp_path / "input_data.csv"
         output_json = tmp_path / "output.json"
         set_params_copy = tmp_path / "set_parameter_filenames.csv"  # Temporary set_parameter_filenames

         try:
             # 1. Create modified input CSV
             original_input = self.scope_dir / "input" / "input_data.csv"
             with open(original_input, 'r') as f:
                 content = f.read()
             modified_input = self._update_input(content, params)  #USE CORRECT FUNCTION
             with open(input_copy, 'w') as f:
                 f.write(modified_input)

             # 2. Modify set_parameter_filenames.csv to point to the temporary input file
             original_set_params = self.scope_dir / "set_parameter_filenames.csv"
             with open(original_set_params, 'r') as f:
                 set_params_content = f.readlines()  # Read as lines

             # Find and replace the input_data.csv filename
             for i, line in enumerate(set_params_content):
                 if "input_data.csv" in line:
                     set_params_content[i] = line.replace("input_data.csv", input_copy.name)
                     break

             with open(set_params_copy, 'w') as f:
                 f.writelines(set_params_content)

             # 3. Copy the modified set_parameter_filenames.csv to the SCOPE directory
             self.eng.copyfile(str(set_params_copy.resolve()), str(original_set_params.resolve()), nargout=0)

             # 4. Run through MATLAB interface
             self.eng.run_scope_wrapper_json(
                 str(input_copy.resolve()),  # Still pass input_copy (though it's now redundant)
                 str(output_json.resolve()),
                 nargout=0
             )

             # 5. Load results
             if not output_json.exists():
                 raise RuntimeError("No output file generated")

             with open(output_json) as f:
                 return json.load(f)
         except Exception as e:
             print(f"Simulation failed: {str(e)}")
             return None
         finally:
             # VERY IMPORTANT: Restore original set_parameter_filenames.csv
             try:
                 self.eng.copyfile(str(set_params_copy.resolve()), str(original_set_params.resolve()), nargout=0)
             except Exception as e:
                 print(f"Error restoring set_parameter_filenames.csv: {e}")

 def _update_input(self, content, params):
     """
     Corrected CSV parameter update. Reads ALL keys from the original,
     updates values, and writes back ONLY valid 'parameter, value' lines.
     """
     lines = []
     original_values = {}

     # 1. Read ALL original keys and values, skipping comments and sections.
     for line in content.split('\n'):
         clean_line = line.strip()
         # Skip comments (%) and section headers ([...])
         if ',' in clean_line and not clean_line.startswith('[') and not clean_line.startswith('%'):
             key_part, value_part = map(str.strip, clean_line.split(',', 1))
             key = key_part.lower().replace(':', '')  # Remove ':' and lowercase
             original_values[key] = value_part

     # 2. Update values based on the provided 'params' dictionary.
     for section, section_params in params.items():
         for key, new_value in section_params.items():
             lower_key = key.lower()
             if lower_key in original_values:
                 original_values[lower_key] = str(new_value)
             #     print(f"Warning: Parameter '{key}' not found in original input_data.csv")

     # 3. Write back ONLY valid 'parameter, value' lines.
     for key, value in original_values.items():
         lines.append(f"{key}, {value}")  # No quotes around the key

     modified_content = '\n'.join(lines)
     print("Modified CSV Content:\n", modified_content)  # Diagnostic print
     return modified_content

In [82]:




# Diagnostic output
print("\n=== Path Diagnostics ===")
print(f"Current working directory: {Path.cwd()}")
print(f"Checking for 'Final_Project/SCOPE' in: {Path.cwd() / 'Final_Project'}")

try:
    test_wrapper = SCOPEWrapper()
    print("\n=== SCOPE Path Verification ===")
    print(f"Using SCOPE directory: {test_wrapper.scope_dir}")
    print("Directory exists:", test_wrapper.scope_dir.exists())
    if test_wrapper.scope_dir.exists():
        print("Contents:", [f.name for f in test_wrapper.scope_dir.glob('*')])
except Exception as e:
    print(f"\nConfiguration error: {str(e)}")

# Test run
if __name__ == "__main__":
    try:
        with SCOPEWrapper() as scope:
            print("\n=== Running Test Simulation ===")
            # Test parameters should match SCOPE's input sections exactly
            # Updated test parameters matching CSV sections
            test_params = {
                "leafbio": {  # Matches [leafbio] section
                    "Cab": 45,
                    "Cca": 25,  # Add required parameters
                    "Cw": 0.02,
                    "N": 1.5
                },
                "canopy": {    # Matches [canopy] section
                    "LAI": 3.2,
                    "hc": 1.5
                },
                "angles": {     # Matches [angles] section
                    "tts": 35,
                    "tto": 0,
                    "psi": 0
                }
            }

            result = scope.run(test_params)
            if result:
                print("Success! First reflectance value:", result['spectrum']['reflectance'][0])
    except Exception as e:
        print(f"\nRuntime error: {str(e)}")



=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'Final_Project/SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/Final_Project

=== SCOPE Path Verification ===
Using SCOPE directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Contents: ['.DS_Store', 'run_scope_wrapper_json.m', 'input', 'output', 'docs', 'soltir_tp7.m', 'SCOPE.m', 'README.md', '.gitignore', 'set_parameter_filenames.csv', 'bug_reports.txt', 'GNU_General_Public_Licence.txt', '.gitattributes', 'run_scope_wrapper.m', '.git', '.readthedocs.yaml', 'SCOPE.exe', 'src']

=== Running Test Simulation ===
Modified CSV Content:
 "cab" 45, 
}, 
"lai" 3.2, 
Simulation failed: File: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/run_scope_wrapper_json.m Line: 7 Column: 8
Invalid use of operator.



File: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/run_scope_wrapper_json.m Line: 7 Column: 8
Invalid use of operator.



In [70]:
from pathlib import Path

# Check current working directory
print(f"Current working directory: {Path.cwd()}")

# Define expected paths
expected_scope_path = Path("Final_Project/SCOPE")

# Check existence
print(f"SCOPE exists: {expected_scope_path.exists()}")
print(f"SCOPE is directory: {expected_scope_path.is_dir()}")
print(f"Contents of Final_Project: {list(Path('Final_Project').glob('*'))}")


Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
SCOPE exists: False
SCOPE is directory: False
Contents of Final_Project: []


In [40]:
def run(self, params):
    """Execute SCOPE simulation with validated parameters"""
    # Validate required parameters
    required_params = ['cab', 'lai', 'sza']
    if missing := [p for p in required_params if p not in params]:
        raise ValueError(f"Missing parameters: {missing}")

    # Create MATLAB structure with nested fields
    mat_params = self.eng.struct()
    
    # Leaf parameters
    mat_params['leafbio'] = self.eng.struct()
    mat_params['leafbio']['Cab'] = matlab.double([params['cab']])
    
    # Canopy parameters
    mat_params['canopy'] = self.eng.struct()
    mat_params['canopy']['LAI'] = matlab.double([params['lai']])
    
    # Observation geometry
    mat_params['angles'] = self.eng.struct()
    mat_params['angles']['tts'] = matlab.double([params['sza']])
    
    # Execute SCOPE
    return self.eng.run_scope_wrapper(mat_params, nargout=2)

In [41]:
with SCOPEWrapper() as scope:
            test_params = {
                'cab': 45,
                'lai': 3.2,
                'sza': 35
            }
            reflectance, fluorescence = scope.run(test_params)
            print(f"Success! Output shapes: {len(reflectance)}, {len(fluorescence)}")

> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
In SCOPE (line 31)
In run_scope_wrapper_json (line 34)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
In SCOPE (line 32)
In run_scope_wrapper_json (line 34)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
In SCOPE (line 33)
In run_scope_wrapper_json (line 34)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
In SCOPE (line 34)
In run_scope_wrapper_json (line 34)
Simulation failed: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/run_scope_wrapper_json.m, line 60, in run_scope_wrapper_json
Reference to a cleared variable backup_path.



Reference to a cleared variable backup_path.

Error in run_scope_wrapper_json (line 60)
        if exist(backup_path, 'file')



TypeError: cannot unpack non-iterable NoneType object

In [ ]:
import numpy as np

# Example parameters
params = {
    'cab': 45,        # Chlorophyll content [μg/cm²]
    'lai': 3.2,       # Leaf Area Index [m²/m²]
    'sza': 35         # Solar Zenith Angle [degrees]
}

with SCOPEWrapper() as scope:
    try:
        reflectance, fluorescence = scope.run(params)
        print("Simulation successful!")
    except Exception as e:
        print(f"Error: {str(e)}")
    
print(f"Reflectance shape: {reflectance.shape}")
print(f"Fluorescence shape: {fluorescence.shape}")


> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
Error: Undefined function 'run_scope_wrapper' for input arguments of type 'struct'.

Reflectance shape: (4001,)
Fluorescence shape: (4001,)


Unrecognized function or variable 'run_scope_wrapper'.



In [ ]:
from joblib import Parallel, delayed

def generate_sample(i):
    """Generate a single sample with proper MATLAB context"""
    with SCOPEWrapper() as scope:
        params = {
            'cab': np.random.uniform(20, 80),
            'lai': np.random.uniform(0.5, 6.0),
            'sza': np.random.uniform(20, 70)
        }
        
        try:
            reflectance, fluorescence = scope.run(params)
            # Save results and process MODTRAN
            # ... rest of your processing ...
        except Exception as e:
            print(f"Error in sample {i}: {str(e)}")

def generate_parallel(num_samples):
    """Run parallel generation with safe MATLAB instances"""
    Parallel(n_jobs=4)(  # Reduce jobs if memory constrained
        delayed(generate_sample)(i)
        for i in range(num_samples)
    )



# Usage
generate_parallel(10)


KeyboardInterrupt: 

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def sample_parameters(ranges):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(np.random.rand(1000, len(ranges)))
    return scaler.inverse_transform(scaled)


In [ ]:
 # --- Define SCOPE input parameters (Example) ---
 #  These need to be converted to MATLAB types.
lai = 3.0
cab = 40.0
tts = 30.0
# % In run_scope_wrapper.m


In [ ]:
 # --- Call SCOPE ---
 #  You'll likely need to call a specific SCOPE function (e.g., 'SCOPE' or a wrapper function you create).
 #  The 'nargout=0' is important if the function doesn't return anything directly.
 #  If the function *does* return values, you'll need to specify the number of output arguments.

 # Example (assuming a function called 'run_scope' that takes LAI, Cab, tts as input):
 #  And returns reflectance ('R') and fluorescence ('F')
# --- Call SCOPE ---
try:
    # Add SCOPE directory to MATLAB path
    eng.addpath(r'SCOPE', nargout=0)
    eng.addpath(r'SCOPE/src', nargout=0)
    eng.addpath(r'SCOPE/input', nargout=0)

    # Create parameter structure
    params = eng.struct()
    params['LAI'] = matlab.double([lai])
    params['Cab'] = matlab.double([cab])
    params['tts'] = matlab.double([tts])

    # Call wrapper
    result = eng.run_scope_wrapper(params, nargout=2)
    reflectance = np.array(result[0]).flatten()
    fluorescence = np.array(result[1]).flatten()

except matlab.engine.MatlabExecutionError as e:
    print(f"MATLAB Error: {e}")
    # Handle error


MATLAB Error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/run_scope_wrapper.m, line 18, in run_scope_wrapper
Execution of script SCOPE as a function is not supported:
/Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/SCOPE.m



Execution of script SCOPE as a function is not supported:
/Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE/SCOPE.m

Error in run_scope_wrapper (line 18)
    [rad, thermal, flux] = SCOPE(soil, leafbio, canopy, atm, angles);



In [ ]:
# --- Clean up ---
eng.quit() # Close the MATLAB session (important!)

In [ ]:
import numpy as np
import scipy.interpolate
from scipy.ndimage import gaussian_filter
# import subprocess # No longer used directly
# import os # No longer needed
from pathlib import Path
import pandas as pd  # For reading MODTRAN output
import matlab.engine  # For interacting with MATLAB


# --- 1. Define Parameter Ranges ---
# *SAME AS BEFORE*, but adjusted for your specific simulation needs

# SCOPE Parameters (Example)
lai_values = np.linspace(0.5, 6, 20)
cab_values = np.linspace(20, 80, 20)
cbrown_values = [0]  # No variation
cw_values = np.linspace(0.008, 0.03, 20)
cm_values = np.linspace(0.004, 0.01, 20)
lidfa_values = [-0.4]  # Spherical leaf angle distribution
lidfb_values = [0.0]  # Spherical leaf angle distribution
tts_values = np.linspace(20, 70, 6)  # Vary solar zenith angle
tto_values = [0]  # Nadir viewing
psi_values = [0]   # We don't need it

# MODTRAN Parameters (Example)
model = 2  # Mid-Latitude Summer
i_aero = 2  # Rural aerosol model
vis_values = np.linspace(5, 50, 10)  # Vary visibility!
gndalt = 0.6 # km.

# HyPlant parameters (same as before)
hyplant_fwhm = 0.27
hyplant_ssi = 0.11
hyplant_wavelengths = np.arange(670, 780, hyplant_ssi)
wavelengths = np.arange(750, 770.001, 0.005) #high-resolution for MODTRAN and SCOPE

# --- 2. Paths (CRUCIAL: Adjust these to your system) ---

modtran_exe_path = Path("./Final_Project/MODTRAN5/bin/Mod5_win64.exe")
modtran_data_path = Path("./Final_Project/MODTRAN5/bin/DATA")  # Path to DATA folder
scope_path = Path("./Final_Project/SCOPE") #Path to the SCOPE folder.
matlab_functions_path = Path('./Final_Project/MODTRAN5/matlab-modtran-5-aba70d781805') # Path to the matlab functions provided.
matlab_t14_path = Path('./Final_Project/MODTRAN5/modtran5_acd_t14') # Path to soltir_tp7.m

# --- 3. Helper Functions (Modified) ---

# Modified to use matlab.engine.
def run_modtran(params_matlab, matlab_engine):
    """
    Runs MODTRAN using the provided parameters.

    Args:
        params_matlab (matlab.engine.struct): A MATLAB structure containing the
            MODTRAN input parameters.
        matlab_engine: The MATLAB engine instance

    Returns:
        tuple: A tuple containing the interpolated solar irradiance and
               atmospheric transfer functions (t1-t12).
    """
    try:
        # Call the MATLAB function run_MODTRAN5_AC_PAR_4RUNS.
        # Make sure the path is correct.
        # nargout=4 is VERY IMPORTANT: it tells MATLAB to return 4 values.
        result = matlab_engine.run_MODTRAN5_AC_PAR_4RUNS(
            './', 'MODTRAN_SCOPE_ATM', params_matlab, 'FLUO', 'A', 1, 0, 0, nargout=4
        )

        # Extract results and convert to NumPy arrays.
        # The .flatten() converts 2D arrays to 1D.
        wvlLUT          = np.array(result[0]).flatten()
        T14            = np.array(result[1])
        DV             = np.array(result[2]).flatten()
        # atmospheric_ch  = np.array(result[3]).flatten() #Not needed.

        # Extract the relevant transfer functions (t1-t12).
        #  Make SURE you're extracting the right columns.
        t1 = T14[:, 0]
        t2 = T14[:, 1]
        t3 = T14[:, 2]
        t4 = T14[:, 3]
        t5 = T14[:, 4]
        t6 = T14[:, 5]
        t7 = T14[:, 6]
        t8 = T14[:, 7]
        t9 = T14[:, 8]
        t10 = T14[:, 9]
        t11 = T14[:, 10]
        # t12 is all zeros
        t12 = np.zeros_like(t1)
        
        # Extract solar irradiance
        solar_irr = T14[:,14] # The solar irradiance should be here
        
        # Interpolate to high-res wavelengths
        t_values_interp = {}  # Store interpolated t values in a dictionary

        f_interp_solar_irr = scipy.interpolate.interp1d(wvlLUT, solar_irr, bounds_error = False, fill_value = 'extrapolate')
        solar_irradiance_interp = f_interp_solar_irr(wavelengths)
        
        for i in range(1, 13):  # t1 to t12
            if i != 12: # Don't interpolate t12
                t_func = scipy.interpolate.interp1d(wvlLUT, T14[:, i-1], bounds_error=False, fill_value='extrapolate') # Extrapolate if needed
                t_values_interp[f't{i}'] = t_func(wavelengths) #interpolate
            else:
               t_values_interp['t12'] = t12


    except matlab.engine.MatlabExecutionError as e:
        print(f"MATLAB Error: {e}")
        return None, None # Return None if MODTRAN fails.

    return solar_irradiance_interp, t_values_interp



def calculate_t_values(solar_irradiance, transmittance_dir, transmittance_dif_down, transmittance_dif_up, path_radiance):
    # THIS FUNCTION IS NO LONGER NEEDED BECAUSE WE DIRECTLY OBTAIN THE t
    # VALUES FROM THE MODTRAN OUTPUT.
    pass

def simulate_hyplant(lat_s, wavelengths, hyplant_wavelengths, hyplant_fwhm, delta_lambda=0, delta_sigma=0):
    # ... (Same as before)
    wavelengths_shifted = wavelengths + delta_lambda
    fwhm_shifted = hyplant_fwhm + delta_sigma
    sigma = fwhm_shifted / (2 * np.sqrt(2 * np.log(2)))
    lat_s_convolved = gaussian_filter(lat_s, sigma / (wavelengths[1] - wavelengths[0]))
    f_interp = scipy.interpolate.interp1d(wavelengths_shifted, lat_s_convolved, bounds_error=False, fill_value=0)
    lhyp = f_interp(hyplant_wavelengths)
    return lhyp



def run_scope(params, eng):
    '''
    Runs the SCOPE model given the parameters.
    
    Args:
    params: A dictionary with the parameters for the model.
    eng: The MATLAB engine instance.
    
    Returns:
    reflectance, fluorescence: The reflectance and fluorescence simulated by SCOPE.
    
    '''
    try:

        # Call SCOPE (replace with your actual SCOPE call)
        # result = eng.run_scope_wrapper(params, nargout=2)  # Adjust nargout
        result = eng.SCOPE(params, nargout=1)  # Call the main SCOPE.m function, and get only one structure output.
        # I assume here that the result is a MATLAB struct
        # containing the fields 'reflectance' and 'fluorescence'.
        # *ADJUST* this if your SCOPE setup returns the data differently.
        # The following two lines *MIGHT* need to be transposed; test this.
        reflectance  = np.array(result['reflectance']).flatten()  # Extract and convert to NumPy array.
        fluorescence = np.array(result['Fluspect']).flatten()    # Extract and convert to NumPy array.
    except matlab.engine.MatlabExecutionError as e:
        print(f"MATLAB Error: {e}")
        # Handle the error appropriately (e.g., log it, skip this simulation)
        reflectance = None  # Or some other default value
        fluorescence = None

    return reflectance, fluorescence
# --- 4. Main Simulation Loop ---
# Start MATLAB engine outside the loop
eng = matlab.engine.start_matlab()

# Add paths to the MATLAB engine
eng.addpath(str(matlab_functions_path), nargout=0)
eng.addpath(str(matlab_t14_path), nargout=0)
eng.addpath(str(scope_path), nargout=0)

# Pre-allocate for results
num_simulations = (len(lai_values) * len(cab_values) * len(cbrown_values) * len(cw_values) *
                    len(cm_values) * len(tts_values) * len(vis_values) * len(lidfa_values) * len(lidfb_values))

num_pixels = 17 * 17
lhyp_data = np.zeros((num_simulations, num_pixels, len(hyplant_wavelengths)))
parameter_data = np.zeros((num_simulations, 9))  # Adjust for all your parameters


simulation_index = 0

for lai in lai_values:
    for cab in cab_values:
        for cbrown in cbrown_values:
            for cw in cw_values:
                for cm in cm_values:
                    for tts in tts_values:
                        for lidfa in lidfa_values:
                            for lidfb in lidfb_values:
                                for vis in vis_values:

                                    # --- Prepare MODTRAN input parameters (as a MATLAB struct) ---
                                    params_modtran = eng.struct()
                                    params_modtran['ATM'] = eng.struct()  # Nested structure
                                    params_modtran['ATM']['LSUNFL'] = 1.0  # Use a specific solar irradiance file
                                    params_modtran['ATM']['MODEL'] = float(model)
                                    params_modtran['ATM']['IHAZE'] = float(i_aero)
                                    params_modtran['ATM']['VIS'] = float(vis)
                                    params_modtran['ATM']['GNDALT'] = float(gndalt)
                                    params_modtran['ATM']['H1'] = 100.0 #Fixed
                                    params_modtran['ATM']['SZA'] = float(tts) # Solar Zenith angle
                                    params_modtran['ATM']['WVL'] = matlab.double([float(wavelengths.min()), float(wavelengths.max())])  # Convert to MATLAB double array

                                    # --- Run MODTRAN ---
                                    solar_irr, t_values = run_modtran(params_modtran, eng)

                                    if solar_irr is None or t_values is None:
                                        print(f"Skipping simulation {simulation_index} due to MODTRAN error.")
                                        simulation_index += 1 # Increment to avoid holes in output arrays.
                                        continue  # Skip to the next iteration


                                    # --- Prepare SCOPE input parameters ---
                                    # Example: You may need to adjust based on your SCOPE setup.
                                    params_scope = eng.struct() # Create the struct.
                                    params_scope['LAI']    = matlab.double([lai])
                                    params_scope['Cab']    = matlab.double([cab])
                                    params_scope['Cbrown'] = matlab.double([cbrown])
                                    params_scope['Cw']     = matlab.double([cw])
                                    params_scope['Cm']     = matlab.double([cm])
                                    params_scope['tts']    = matlab.double([tts])
                                    params_scope['tto']    = matlab.double([tto_values[0]]) # tto and psi are fixed.
                                    params_scope['psi']    = matlab.double([psi_values[0]])
                                    params_scope['lidfa']  = matlab.double([lidfa])
                                    params_scope['lidfb']  = matlab.double([lidfb])

                                    # --- Run SCOPE ---
                                    reflectance, fluorescence = run_scope(params_scope, eng)
                                    if reflectance is None or fluorescence is None:
                                        print(f"Skipping simulation {simulation_index} due to SCOPE error.")
                                        simulation_index += 1
                                        continue

                                    # Interpolate SCOPE output to MODTRAN wavelengths
                                    f_interp_R = scipy.interpolate.interp1d(np.arange(400, 2501), reflectance, bounds_error=False, fill_value='extrapolate')
                                    reflectance_interp = f_interp_R(wavelengths)

                                    f_interp_F = scipy.interpolate.interp1d(np.arange(400, 2501), fluorescence, bounds_error=False, fill_value='extrapolate')
                                    fluorescence_interp = f_interp_F(wavelengths)


                                    # --- Calculate At-Sensor Radiance (Lat-s) ---
                                    r_s = reflectance_interp  # Reflectance of the target
                                    f_s = fluorescence_interp  # Fluorescence of the target
                                    r_surr = r_s  # Homogeneous assumption
                                    f_surr = f_s  # Homogeneous assumption
                                    e_0 = solar_irr * np.cos(np.deg2rad(tts))  # Modulate by solar zenith angle

                                    l_r = e_0 * (t_values['t1'] + t_values['t7'] * r_s + t_values['t8'] * r_surr + t_values['t10'] * r_surr + t_values['t9'] * r_s) / (1 - t_values['t2'] * r_surr)
                                    l_f = t_values['t5'] * (f_s + (t_values['t6'] + t_values['t11'] * r_surr) * f_surr) / (1-t_values['t2'] * r_surr) # Bayat (2020)
                                    lat_s = l_r / np.pi + l_f

                                    # --- Simulate HyPlant Observation ---
                                    lhyp = simulate_hyplant(lat_s, wavelengths, hyplant_wavelengths, hyplant_fwhm)
                                    # Create a 17x17 patch (homogeneous in this example)
                                    lhyp_patch = np.tile(lhyp, (num_pixels, 1))

                                    # --- Store Results ---
                                    lhyp_data[simulation_index, :, :] = lhyp_patch
                                    parameter_data[simulation_index, :] = [lai, cab, cbrown, cw, cm, lidfa, lidfb, tts, vis]

                                    simulation_index += 1

# --- 5. Save Data ---
np.savez('synthetic_dataset.npz', lhyp=lhyp_data, parameters=parameter_data)
# Close MATLAB engine when finished
eng.quit()

print("Simulations Complete!")

> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)
> In path (line 109)
In addpath>doPathAddition (line 116)
In addpath (line 80)


Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.



MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 0 due to MODTRAN error.


Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.



MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 2 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 3 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 4 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 5 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 6 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 7 due to MODTRAN

...

MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1007 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1008 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1009 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1010 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1011 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1012 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulatio

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_M

MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1026 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1027 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1028 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1029 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1030 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulation 1031 due to MODTRAN error.
MATLAB Error: Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.

Skipping simulatio

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Unrecognized function or variable 'run_MODTRAN5_AC_PAR_4RUNS'.

Undefined function 'run_MODTRAN5_AC_PAR_4RUNS' for input arguments of type 'struct'.



TypeError: 'NoneType' object is not subscriptable